# Skin Atlas Glycomics Data Analysis Notebook

The below script contains code for the analysis of 3 glycomics samples generated for the skin cancer atlas publication. 
* code below is finalised analysis


Author: Andrew Causer (6/1/2025)

## Library and Data Loading

#### Required Libraries 

In [ ]:
library(SpaMTP)
library(Seurat)
library(Cardinal)
library(ggplot2)
library(viridis)
library(EnhancedVolcano)

In [ ]:
setwd("/scratch/user/uqacause/files/GML_SkinAtlas/data/raw_data/")

#### Set I/O Directories

In [ ]:
## Data must be symbolically linked from RDM/Scratch (check out README.md file for more infomation on how to link files)
## or INPUT_DIR can be changed to define the path to data stored elsewhere.

INPUT_DIR <- "../data/raw_data/"
OUTPUT_DIR <- "../data/output/"

In [ ]:
# Create the directory if it doesn't exist
if (!dir.exists(OUTPUT_DIR)) {
  dir.create(OUTPUT_DIR)
  message("Directory 'output' created.")
} else {
  message("Directory 'output' already exists.")
}

#### Set Sample Names for Analysis

In [ ]:
# Get sample names
sample_names <- list.dirs("/scratch/user/uqacause/files/GML_SkinAtlas/data/raw_data/", recursive = F, full.names = F)

print(sample_names)

In [ ]:
# Create list to store data 
data_list <- list()

These sample names will be used to load/store the processed data for each sample

## Read in data using *SpaMTP*

In [ ]:
for (sample in sample_names) {
    data_list[[sample]] <- LoadSM(name = sample, path = paste0("/scratch/user/uqacause/files/GML_SkinAtlas/data/raw_data/", "/", sample, "/"), mass.range = c(900, 3500), resolution = 250, units = "ppm")
}

Below is a printout of the loaded data. Each sample has 5433 m/z values and contain roughly between 66,000 and 170,000 pixels each.

In [ ]:
saveRDS(data_list, "/QRISdata/Q4427/glycomic_data/spamtp_data_objects.RDS")

## Individual Sample Analysis

Each sample will be analysed indiviudally following a standard pipeline:
1. Annotate the m/z values using SpaMTP against the LIPIDMAPS database
2. Run PCA and clustering analysis using 30pc's and a resolution of 0.3
3. Remove background noise from sample (outside of tissue) and rerun clustering
4. Based on histological analysis identify the cluster which matches the cancer region
5. Run differential metabolite abundance analysis to identify metabolites which are significantly up regulated by this cancer cluster

#### Run Clustering Per Sample 

In [ ]:
## Annotate Each Sample
for (sample in names(data_list)){
    data_list[[sample]] <- AnnotateSM(data_list[[sample]], db = Lipidmaps_db, ppm_error = 5, polarity = "positive", return.only.annotated = F)
}

In [ ]:
## Run PCA for each sample
for (sample in names(data_list)){
    data_list[[sample]] <- RunMetabolicPCA(SpaMTP = data_list[[sample]], assay = "Spatial")
}

In [ ]:
## Cluster Each Sample
for (sample in names(data_list)){
    data_list[[sample]] <- FindNeighbors(data_list[[sample]], dims = 1:30, verbose = FALSE) # use the SpaMTP object generated from PCA Pathway Analysis
    data_list[[sample]] <- RunUMAP(data_list[[sample]], dims = 1:30, verbose = FALSE)
    data_list[[sample]] <- FindClusters(data_list[[sample]], resolution = 0.3, cluster.name = "clusters_per_sample", verbose = FALSE)
}

In [ ]:
## Plot each sample
plots <- list()
for (sample in names(data_list)){
    if (sample != "sample1"){
        plots[[sample]] <- ImageDimPlot(data_list[[sample]], size = 4) + theme_void()+ coord_flip()+ggplot2::scale_x_reverse()
    } else {
        plots[[sample]] <- ImageDimPlot(data_list[[sample]], size = 3)+ theme_void()
    }
}

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

plots[[1]]
plots[[2]]
plots[[3]]

Based on the plots above we can see that there is some background outside the tissue. We will remove that and then continue the analysis

In [ ]:
data_subset <- list()
data_subset[["sample1"]] <- subset(data_list[["sample1"]], subset = clusters_per_sample %in% c(4, 6, 10, 12), invert = T)
data_subset[["sample2"]] <- subset(data_list[["sample2"]], subset = clusters_per_sample %in% c(4, 10, 12), invert = T)
data_subset[["sample3"]] <- subset(data_list[["sample3"]], subset = clusters_per_sample %in% c(1, 11, 8), invert = T)

In [ ]:
## Plot each sample
plots <- list()
for (sample in names(data_subset)){
    if (sample != "sample1"){
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 4) + theme_void()+ coord_flip()+ggplot2::scale_x_reverse()
    } else {
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 3)+ theme_void()
    }
}

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

plots[[1]]
plots[[2]]
plots[[3]]

In [ ]:
## Run PCA for each sample
for (sample in names(data_subset)){
    data_subset[[sample]] <- RunMetabolicPCA(SpaMTP = data_subset[[sample]], assay = "Spatial")
}

In [ ]:
## Cluster Each Sample
for (sample in names(data_subset)){
    data_subset[[sample]] <- FindNeighbors(data_subset[[sample]], dims = 1:30, verbose = FALSE) # use the SpaMTP object generated from PCA Pathway Analysis
    data_subset[[sample]] <- RunUMAP(data_subset[[sample]], dims = 1:30, verbose = FALSE)
    data_subset[[sample]] <- FindClusters(data_subset[[sample]], resolution = 0.3, cluster.name = "clusters_per_sample_filtered", verbose = FALSE)
}

In [ ]:
## Plot each sample
plots <- list()
for (sample in names(data_subset)){
    if (sample != "sample1"){
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 4, group.by = "clusters_per_sample_filtered") & theme_void()& coord_flip()&ggplot2::scale_x_reverse()|
                            ImageDimPlot(data_subset[[sample]], size = 4, group.by = "clusters_per_sample") & theme_void()& coord_flip()&ggplot2::scale_x_reverse()
    } else {
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 3, group.by = "clusters_per_sample_filtered") & theme_void()| 
                           ImageDimPlot(data_subset[[sample]], size = 3, group.by = "clusters_per_sample") & theme_void()
    }
}

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)

plots[[1]]
plots[[2]]
plots[[3]]

#### Identify Melanoma region

In [ ]:
## Plot each sample
plots <- list()
for (sample in names(data_subset)){
    if (sample != "sample1"){
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 3, split.by = "clusters_per_sample_filtered", dark.background = F)& coord_flip()&ggplot2::scale_x_reverse() | 
        ((ImageDimPlot(data_subset[[sample]], size = 3, group.by = "clusters_per_sample_filtered")& theme_void()& coord_flip()&ggplot2::scale_x_reverse() )/ DimPlot(data_subset[[sample]], group.by = "clusters_per_sample_filtered"))
    } else {
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 2, split.by = "clusters_per_sample_filtered", dark.background = F)  |
        ((ImageDimPlot(data_subset[[sample]], size = 2, group.by = "clusters_per_sample_filtered")& theme_void()) /DimPlot(data_subset[[sample]], group.by = "clusters_per_sample_filtered"))
    }
}

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)

for (sample in names(plots)){
    print(plots[[sample]])
    #ggplot2::ggsave(plots[[sample]], filename = paste0(OUTPUT_DIR, sample, "_individual_clustering_results.png"), height = 10, width = 30, dpi = 500)
}

Based on the plots above the melanoma clusters for each sample are:
* Sample1 = cluster 6
* Sample2 = cluster 8
* Sample3 = cluster 6

In [ ]:
melanoma_clusters <- list("6", "8", "6")
names(melanoma_clusters) <- names(data_subset)

## Identify Common Metabolites associated with Melanoma Region 

First we will identify DE genes that are differentially expressed in each of the melanoma clusters

In [ ]:
whole_de <- list()


for (sample in names(data_subset)){
    ## Plot each sample
    whole_de[[sample]] <- FindAllDEMs(data_subset[[sample]], ident = "clusters_per_sample_filtered", return.individual = F) 
}



In [ ]:
merged_de <- whole_de

In [ ]:
merged_de[[1]]$DEMs <- NULL
merged_de[[2]]$DEMs <- NULL
merged_de[[3]]$DEMs <- NULL

In [ ]:

# Add prefixes to sample names in each DGEList object
colnames(merged_de[[1]]$counts) <- paste0("s1_", colnames(merged_de[[1]]$counts))
colnames(merged_de[[2]]$counts) <- paste0("s2_", colnames(merged_de[[2]]$counts))
colnames(merged_de[[3]]$counts) <- paste0("s3_", colnames(merged_de[[3]]$counts))

merged_de[[1]]$samples$samp <- "Sample 1"
merged_de[[2]]$samples$samp <- "Sample 2"
merged_de[[3]]$samples$samp <- "Sample 3"



# Merge the counts matrices from all DGEList objects
dge_counts <- cbind(
  merged_de[[1]]$counts,
  merged_de[[2]]$counts,
  merged_de[[3]]$counts
)

# If needed, you can also handle the `samples` data
dge_samples <- rbind(
  merged_de[[1]]$samples,
  merged_de[[2]]$samples,
  merged_de[[3]]$samples
)

# Create a new DGEList object with merged counts and samples
dge <- edgeR::DGEList(counts = dge_counts, samples = dge_samples)


In [ ]:
significant_mz <- list()

for (sample in names(whole_de)){
    ## Plot each sample
    first_10_genes <- whole_de[[1]]$DEMs[whole_de[[1]]$DEMs$regulate != "Normal",] %>%
                      group_by(cluster) %>%          # Group by cluster
                      slice_head(n = 10) %>%         # Select the first 10 rows for each cluster
                      ungroup()      
    significant_mz[[sample]] <- first_10_genes$gene
}

In [ ]:
edgeR_output <- dge
n = 5
 only.pos = FALSE
 FDR.threshold = 0.05
 logfc.threshold = 0.5
 order.by = "FDR"
 scale ="row"
 color = grDevices::colorRampPalette(c("navy", "white", "red"))(50)
 cluster_cols = T
 cluster_rows = T
 fontsize_row = 15
 fontsize_col = 15
 cutree_cols = 9
 silent = TRUE
 plot_annotations_column = NULL
 save_to_path = NULL
 plot.save.width = 20
 plot.save.height = 20
 nlabels.to.show = NULL
 annotation_colors = NULL

  col_annot <- data.frame(sample = edgeR_output$samples$samp)
  row.names(col_annot) <- colnames(as.data.frame(edgeR::cpm(edgeR_output,log=TRUE)))

  if (!is.null(annotation_colors)){
    annotation_colors <- list(sample = unlist(annotation_colors))
  } else {
    annotation_colors <- NA
  }

  mtx <- as.matrix(as.data.frame(edgeR::cpm(edgeR_output,log=TRUE))[unique(unlist(significant_mz)),])
  if (!(is.null(plot_annotations_column))){
    if (is.null(edgeR_output$DEMs[[plot_annotations_column]])){
      warning("There are no annotations present in the edgeR_output object. Run 'annotate.SeuratMALDI()' prior to 'FindAllDEMs' and set annotations = TRUE .....\n Heatmap will plot default m/z values ... ")
    } else{
      if (!is.null(nlabels.to.show)){
        df[[plot_annotations_column]] <- labels_to_show(df[[plot_annotations_column]], n = nlabels.to.show)
      }
      rownames(mtx) <- unique(df[[plot_annotations_column]])
    }
  }
    
  p <- pheatmap::pheatmap(mtx,scale=scale,color=color,cluster_cols = cluster_cols, annotation_col=col_annot, cluster_rows = cluster_rows,
                          fontsize_row = fontsize_row, fontsize_col = fontsize_col, cutree_cols = cutree_cols, silent = silent, annotation_colors = annotation_colors)


In [ ]:
colnames(mtx) <- unlist(lapply(colnames(mtx), function(x) { st <- strsplit(x, split = "_")[[1]]
                                    label <- paste0(st[3], "_", st[4])
                                   label}))

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 12)
p

In [ ]:
pdf(paste0(OUTPUT_DIR,"combined_metabolite_de.pdf"), width=30, height=12)
grid::grid.newpage()
grid::grid.draw(p$gtable)
dev.off()

In [ ]:
DE_results <- list()

for (sample in names(data_subset)){
    ## Plot each sample
    de <- FindAllDEMs(data_subset[[sample]], ident = "clusters_per_sample_filtered", return.individual = T) 
    names(de) <- unlist(lapply(1:length(de), function (x){unique(de[[x]]$DEMs$cluster)}))
    melanoma_de <- de[[melanoma_clusters[[sample]]]]
    melanoma_de$DEMs$cluster <- "Melanoma"
    DE_results[[sample]] <- melanoma_de
}

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
DEMsHeatmap(DE_results[["sample1"]], n = 100, only.pos = F)

Now we can identify metabolites which are up regulated across all 3 samples

In [ ]:
UP_metabolites <- list()

for (sample in names(DE_results)){
    ## Plot each sample
    df <- DE_results[[sample]]$DEMs
    df <- df[df$regulate == "Up",]
    UP_metabolites[[sample]] <- df$gene
}

In [ ]:
common_melanoma_markers <- Reduce(intersect, UP_metabolites)

In [ ]:
cat(paste0(" There were ", length(common_melanoma_markers), "which are named:"))
common_melanoma_markers

## Spatially Plot the common metabolites

In [ ]:
## Adding metadata annotations for only melanoma spots

for (sample in names(data_subset)){
    data_subset[[sample]]@meta.data$melanoma <- ifelse(data_subset[[sample]]@meta.data$clusters_per_sample_filtered == melanoma_clusters[[sample]], "melanoma", "other")
}

In [ ]:
## Plot each sample

spatial_plots <- list()
dim_plots <- list()

for (sample in names(data_subset)){
    if (sample != "sample1"){
        spatial_plots[[sample]] <- ImageFeaturePlot(data_subset[[sample]], features = common_melanoma_markers) & theme_void()& coord_flip()&ggplot2::scale_x_reverse()&scale_fill_gradientn(colors = inferno(100))
    } else {
        spatial_plots[[sample]] <- ImageFeaturePlot(data_subset[[sample]], features = common_melanoma_markers)& theme_void()&scale_fill_gradientn(colors = inferno(100))
    }
    
    dim_plots[[sample]] <- FeaturePlot(data_subset[[sample]], features = common_melanoma_markers, ncol = 5, slot = "counts")&scale_colour_gradientn(colors = inferno(100))
    ggsave(dim_plots[[sample]], filename = paste0(OUTPUT_DIR, sample, "_melanoma_markers_UMAP.png"), height = 20, width = 25, dpi = 500)
    ggsave(spatial_plots[[sample]], filename = paste0(OUTPUT_DIR, sample, "_melanoma_markers_spatial.png"), height = 12, width = 35, dpi = 500)
        
}

In [ ]:
options(repr.plot.width = 35, repr.plot.height = 12)
spatial_plots[[1]]

options(repr.plot.width = 25, repr.plot.height = 20)
dim_plots[[1]]

In [ ]:
plots <- list()

mz <- common_melanoma_markers[1]

for (sample in names(data_subset)){
    if (sample != "sample1"){
        plots[[sample]] <- ImageFeaturePlot(data_subset[[sample]], features = mz, size = 3) & theme_void()& coord_flip()&ggplot2::scale_x_reverse()&scale_fill_gradientn(colors = inferno(100))|
                            ImageDimPlot(data_subset[[sample]], size = 3, group.by = "melanoma", cols = list("melanoma" = "red", "other" = "grey"))& theme_void()& coord_flip()&ggplot2::scale_x_reverse()
    } else {
        plots[[sample]] <- ImageFeaturePlot(data_subset[[sample]], features = mz, size = 2)& theme_void()&scale_fill_gradientn(colors = inferno(100))|
                            ImageDimPlot(data_subset[[sample]], size = 2, group.by = "melanoma", cols = list("melanoma" = "red", "other" = "grey"))& theme_void()
    }
}

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)

for (sample in names(plots)){
    print(plots[[sample]])
    #ggplot2::ggsave(plots[[sample]], filename = paste0(OUTPUT_DIR, sample, "_individual_clustering_results.png"), height = 10, width = 30, dpi = 500)
}

In [ ]:
common_melanoma_markers = c('mz-1583.89893435206', 'mz-1585.08730414198', 'mz-1744.79921812841', 'mz-1746.98158085498', 'mz-1422.81810614908', 'mz-1421.75139263149', 'mz-1581.92029757944', 'mz-2068.11897965862', 'mz-1746.10830839632', 'mz-1743.92703657837', 'mz-1909.11443559991', 'mz-1908.16011697668', 'mz-1583.10718283514', 'mz-2069.1532977118', 'mz-1743.05529101009', 'mz-1907.20627539349', 'mz-1745.23547246474', 'mz-1906.25291061188', 'mz-1584.29495858868', 'mz-1905.30002239352', 'mz-2070.18813305333', 'mz-1582.31582709502', 'mz-1421.04069462078', 'mz-1422.10687491493', 'mz-1419.97531366206', 'mz-1258.77557936752', 'mz-1744.36307284204', 'mz-1705.1268173896')

In [ ]:
melanoma_marker_annotations <- data_subset[[1]]@assays$Spatial@meta.data[data_subset[[1]]@assays$Spatial@meta.data$mz_names %in% common_melanoma_markers,]
melanoma_marker_annotations

In [ ]:
write.csv(melanoma_marker_annotations, paste0(OUTPUT_DIR, "common_melanoma_metabolites.csv"))

In [ ]:
saveRDS(data_subset, paste0(OUTPUT_DIR, "processed_data_objects.RDS"))

In [ ]:
UP_metabolites

In [ ]:
library(ggVennDiagram)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
p <- ggVennDiagram(UP_metabolites) + scale_fill_distiller(palette = "Reds", direction = 1)
p

#ggplot2::ggsave(p, filename = paste0(OUTPUT_DIR, "_upregulated_metabolite_vendiagram.pdf"), height = 10, width = 10)

## Publication Images

In [ ]:
data_subset <- readRDS(paste0(OUTPUT_DIR, "processed_data_objects.RDS"))

In [ ]:
## Plot each sample
p1 <- ImageDimPlot(data_subset[[1]], size = 3, split.by = "clusters_per_sample_filtered", dark.background = F, cols = list()& coord_flip()&ggplot2::scale_x_reverse() | 
        ((ImageDimPlot(data_subset[[sample]], size = 3, group.by = "clusters_per_sample_filtered")& theme_void()& coord_flip()&ggplot2::scale_x_reverse() )/ DimPlot(data_subset[[sample]], group.by = "clusters_per_sample_filtered"))
    } else {
        plots[[sample]] <- ImageDimPlot(data_subset[[sample]], size = 2, split.by = "clusters_per_sample_filtered", dark.background = F)  |
        ((ImageDimPlot(data_subset[[sample]], size = 2, group.by = "clusters_per_sample_filtered")& theme_void()) /DimPlot(data_subset[[sample]], group.by = "clusters_per_sample_filtered"))
    }
}

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
ImageDimPlot(data_subset[[1]], size = 2, split.by = "clusters_per_sample_filtered", dark.background = F, cols = list("0" = "#6e246e",
                                                                                                                    "1" = "#db86db",
                                                                                                                    "2" = "#e0bcf7",
                                                                                                                    "3" = "#bcdef7",
                                                                                                                    "4" = "#2594e6",
                                                                                                                    "5" = "#8659a5",
                                                                                                                    "6" = "#9cd089",
                                                                                                                    "7" = "#d92a28",
                                                                                                                    "8" = "#e61ce6"))

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p <- ImageDimPlot(data_subset[[1]], size = 1.6, group.by = "clusters_per_sample_filtered", dark.background = F, cols = list("0" = "#6e246e",
                                                                                                                    "1" = "#db86db",
                                                                                                                    "2" = "#e0bcf7",
                                                                                                                    "3" = "#bcdef7",
                                                                                                                    "4" = "#2594e6",
                                                                                                                    "5" = "#8659a5",
                                                                                                                    "6" = "#9cd089",
                                                                                                                    "7" = "#d92a28",
                                                                                                                    "8" = "#e61ce6"))&theme_void()
p
ggsave(p, filename = paste0(OUTPUT_DIR, "sample1_spatial_clusters.png"), height = 10, width = 20, dpi = 1000)

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 10)
p <- ImageDimPlot(data_subset[[2]], size = 1.2, group.by = "clusters_per_sample_filtered",  dark.background = F, cols = list("0" = "#6e246e",
                                                                                                                    "1" = "#db86db",
                                                                                                                    "2" = "#198a17",
                                                                                                                    "3" = "#bcdef7",
                                                                                                                    "4" = "#2594e6",
                                                                                                                    "5" = "#8659a5",
                                                                                                                    "6" = "grey",
                                                                                                                    "7" = "#edeb6b",
                                                                                                                    "8" = "#9cd089",
                                                                                                                      "9" = "blue",
                                                                                                                      "10" = "#d92a28",
                                                                                                                      "11" = "#e0bcf7",
                                                                                                                      "12" = "#e61ce6"))& theme_void()& coord_flip()&ggplot2::scale_x_reverse()
p
ggsave(p, filename = paste0(OUTPUT_DIR, "sample2_spatial_clusters.png"), height = 10, width = 25, dpi = 1000)

In [ ]:
ImageDimPlot(data_subset[[3]], size = 1.4, split.by = "clusters_per_sample_filtered", dark.background = F, cols = list("0" = "#6e246e",
                                                                                                                    "1" = "#db86db",
                                                                                                                    "2" = "#198a17",
                                                                                                                    "3" = "#bcdef7",
                                                                                                                    "4" = "#d92a28",
                                                                                                                    "5" = "#8659a5",
                                                                                                                    "6" = "#9cd089",
                                                                                                                    "7" = "#2594e6",
                                                                                                                    "8" = "#e61ce6"))& theme_void()& coord_flip()&ggplot2::scale_x_reverse()

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 10)
p <- ImageDimPlot(data_subset[[3]], size = 1.3, group.by = "clusters_per_sample_filtered", dark.background = F, cols = list("0" = "#6e246e",
                                                                                                                    "1" = "#db86db",
                                                                                                                    "2" = "#f0788c",
                                                                                                                    "3" = "#2594e6",
                                                                                                                    "4" = "#d92a28",
                                                                                                                    "5" = "#8659a5",
                                                                                                                    "6" = "#9cd089",
                                                                                                                    "7" = "#bcdef7",
                                                                                                                    "8" = "#e61ce6"))& theme_void()& coord_flip()&ggplot2::scale_x_reverse() &  scale_shape_manual(values=c(15))
p
ggsave(p, filename = paste0(OUTPUT_DIR, "sample3_spatial_clusters.png"), height = 10, width = 25, dpi = 1000)

In [ ]:
annotated_mzs <- melanoma_marker_annotations[melanoma_marker_annotations$all_IsomerNames != "No Annotation",]$mz_names
melanoma_marker_annotations[melanoma_marker_annotations$all_IsomerNames != "No Annotation",]

In [ ]:
which(annotated_mzs == annotated_mzs[c(2)])

In [ ]:
## Plot each sample

spatial_plots <- list()
dim_plots <- list()

for (sample in names(data_subset)){
    p <- list()
    if (sample != "sample1"){
        for (mz in annotated_mzs[c(2, 3, 6, 7)]){
            p[[mz]] <- ImageFeaturePlot(data_subset[[sample]], features = mz) & theme_void()& coord_flip()&ggplot2::scale_x_reverse()&scale_fill_gradientn(colors = inferno(100))
            ggsave(p[[mz]], filename = paste0(OUTPUT_DIR, sample,"_mz_", which(annotated_mzs == mz), "_melanoma_markers_spatial.png"),height = 5, width = 10, dpi = 1200)
        }
        
    } else {
        for (mz in annotated_mzs[c(2,3, 6, 7)]){
            p[[mz]] <- ImageFeaturePlot(data_subset[[sample]], features = mz)& theme_void()&scale_fill_gradientn(colors = inferno(100))
            ggsave(p[[mz]], filename = paste0(OUTPUT_DIR, sample,"_mz_", which(annotated_mzs == mz), "_melanoma_markers_spatial.png"),height = 5, width = 10, dpi = 1200)
        }
    }
    spatial_plots[[sample]] <- Reduce("|", p)
    #dim_plots[[sample]] <- FeaturePlot(data_subset[[sample]], features = annotated_mzs, ncol = 5, slot = "counts")&scale_colour_gradientn(colors = inferno(100))
    #ggsave(dim_plots[[sample]], filename = paste0(OUTPUT_DIR, sample, "_melanoma_markers_UMAP.png"), height = 20, width = 25, dpi = 500)
    #ggsave(spatial_plots[[sample]], filename = paste0(OUTPUT_DIR, sample, "_melanoma_markers_spatial.png"), height = 12, width = 35, dpi = 500)
        
}

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 5)
spatial_plots[[1]]
ggsave(spatial_plots[[1]], filename = paste0(OUTPUT_DIR, "sample1_melanoma_markers_spatial.pdf"), height = 5, width = 30, dpi = 1000)
options(repr.plot.width = 30, repr.plot.height = 4)
spatial_plots[[2]]
ggsave(spatial_plots[[2]], filename = paste0(OUTPUT_DIR, "sample2_melanoma_markers_spatial.pdf"), height = 4, width = 30, dpi = 1000)
options(repr.plot.width = 30, repr.plot.height = 4)
spatial_plots[[3]]
ggsave(spatial_plots[[3]], filename = paste0(OUTPUT_DIR, "sample3_melanoma_markers_spatial.pdf"), height = 4, width = 30, dpi = 1000)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)

p[[1]]

In [ ]:
sessionInfo()